# Code

In [ ]:
import pandas as pd
import numpy as np
import math
from copy import deepcopy
from random import randint

In [1]:
class KMeans:
    __valid_distance_method = ['manhattan', 'euclidian']
    __valid_init_method = ['random', 'distribute']
    
    def __init__(self, n_clusters = 2, max_iter = 300, distance_method = 'euclidian', init = 'random'):
        if (distance_method not in self.__valid_distance_method):
            raise ValueError("Unknown distance calculation type type %s."
                             "Valid options are %s" % (distance,
                                                       self.__valid_distance_method))
        if (n_clusters < 1):
            raise ValueError("Invalid value of n_clusters, got %d. Must be at least 1" % n_clusters)
        self.__n_clusters = n_clusters
        self.__max_iter = max_iter
        self.__distance_method = distance_method
        self.__init_method = init
        
    def fit(self, data):
        if (len(data) < 2):
            raise ValueError("Need at least 2 instance of data")
        # TODO: check type(data)
        self.__data = data.values.tolist()
        self.__n_data = len(data)
        self.__attr = data.columns.tolist()
        self.__n_attr = len(self.__attr)
        self.labels_ = np.array([])
        self.cluster_centers_ = []
        self.initial_cluster_centers_idx_ = []
        
        if self.__init_method == 'random':
            # randomly select k instance for initial cluster
            for i in range(self.__n_clusters):
                idx = randint(0, self.__n_data)
                while (idx in self.initial_cluster_centers_idx_):
                    idx = randint(0, self.__n_data)
                self.initial_cluster_centers_idx_.append(idx)
                self.cluster_centers_.append(self.__data[idx])
        else:
            section = int(self.__n_data / self.__n_clusters)
            current_idx = 0
            for i in range(self.__n_clusters):
                self.initial_cluster_centers_idx_.append(current_idx)
                self.cluster_centers_.append(self.__data[current_idx])
                current_idx += section
            
        self.__is_convergence = False
        self.__clusters_items = []
        iter_count = 0
        while (not self.__is_convergence) and iter_count < self.__max_iter:
            self.labels_ = []
            self.__clusters_items = [[] for _ in xrange(self.__n_clusters)]
            
            # iterate through all data, and calculate its distance with current centroid
            for i in range(self.__n_data):
                clusters_distance = []
                for center in self.cluster_centers_:
                    clusters_distance.append(
                        self.__calculate_distance(self.__data[i], center, self.__distance_method)
                    )

                cluster = np.argmin(clusters_distance)
                self.labels_.append(cluster)
                self.__clusters_items[cluster].append(self.__data[i])
                

            old_cluster_centers = deepcopy(self.cluster_centers_)
            # calculate new means per cluster
            for i in range(self.__n_clusters):
                cluster_attr_sum = np.array([0] * self.__n_attr)
                n_member = len(self.__clusters_items[i])
                
                if n_member == 0:
                    self.cluster_centers_[i] = self.__data[randint(0, self.__n_data)]
                    continue
                
                for item in self.__clusters_items[i]:
                    for j, attr in enumerate(item):
                        cluster_attr_sum[j] += attr
                
                for j in range(self.__n_attr):
                    cluster_attr_sum[j] = float(float(cluster_attr_sum[j]) / float(n_member))
            
            
            # check if convergence
            old = [item for sublist in old_cluster_centers for item in sublist]
            new = [item for sublist in self.cluster_centers_ for item in sublist]
            if (old[1:] == new[:-1]):
                self.__is_convergence = True
                    
            iter_count += 1
            
        return self
    
    def __calculate_distance(self, X, Y, method = 'euclidian'):
        if (len(X) != len (Y)):
            raise ValueError("Can't calculate distance with across dimension")
        
        if method == 'manhattan':
            return self.__calculate_distance_manhattan(X,Y)
        
        return self.__calculate_distance_euclidian(X,Y)
            
    def __calculate_distance_euclidian(self, X, Y):
        square_sum = 0
        for i in range(len(X)):
            square_sum += (X[i]-Y[i]) ** 2
        return math.sqrt(square_sum)
    
    def __calculate_distance_manhattan(self, X, Y):
        distance_sum = 0;
        for i in range(len(X)):
            distance_sum += abs(X[i]-Y[i])
        return distance_sum;
        

# Contoh pemakaian

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

iris = pd.read_csv('data/Iris.csv')
iris = iris.drop(labels='Id', axis=1)

iris_data = iris.iloc[:,:-1]
iris_label = iris.iloc[:,-1]
species_encoder = LabelEncoder().fit(iris_label)
iris_label_encoded = species_encoder.transform(iris_label)

In [3]:
iris_data.columns.tolist()

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

## Inisiasi random

In [4]:
for _ in range(5):
    kmeans = KMeans(n_clusters = 3, distance_method = 'euclidian', init='random').fit(iris_data)
    mat = confusion_matrix(kmeans.labels_, iris_label_encoded)
    purity = float(mat[0].max() + mat[1].max() + mat[2].max()) / float(mat.sum())
    
    print pd.crosstab(iris_label, np.array(kmeans.labels_))
    print kmeans.cluster_centers_
    print "Purity: ", purity, "\n"

col_0             0   1   2
Species                    
Iris-setosa      10   0  40
Iris-versicolor   0  47   3
Iris-virginica    0  50   0
[[5.8, 4.0, 1.2, 0.2], [6.8, 2.8, 4.8, 1.4], [4.7, 3.2, 1.6, 0.2]]
Purity:  0.666666666667 

col_0             0  1   2
Species                   
Iris-setosa      47  3   0
Iris-versicolor  40  9   1
Iris-virginica   12  1  37
[[6.1, 2.9, 4.7, 1.4], [6.3, 3.3, 4.7, 1.6], [6.5, 3.0, 5.5, 1.8]]
Purity:  0.62 

col_0             0   1   2
Species                    
Iris-setosa      50   0   0
Iris-versicolor   0  27  23
Iris-virginica    0   2  48
[[5.5, 3.5, 1.3, 0.2], [4.9, 2.5, 4.5, 1.7], [6.1, 2.6, 5.6, 1.4]]
Purity:  0.833333333333 

col_0            0   1   2
Species                   
Iris-setosa      9  41   0
Iris-versicolor  0   4  46
Iris-virginica   0   0  50
[[5.8, 4.0, 1.2, 0.2], [4.8, 3.4, 1.6, 0.2], [6.7, 3.0, 5.0, 1.7]]
Purity:  0.666666666667 

col_0             0   1   2
Species                    
Iris-setosa      50   0   0
Iris

In [5]:
pd.crosstab(iris_label, np.array(kmeans.labels_))

col_0,0,1,2
Species,,,
Iris-setosa,50,0,0
Iris-versicolor,0,48,2
Iris-virginica,0,12,38


In [6]:
kmeans.cluster_centers_

[[5.0, 3.5, 1.3, 0.3], [6.3, 2.3, 4.4, 1.3], [6.7, 3.3, 5.7, 2.1]]

## Inisiasi distribute

In [7]:
kmeans = KMeans(n_clusters = 3, distance_method = 'euclidian', init='distribute').fit(iris_data)
mat = confusion_matrix(kmeans.labels_, iris_label_encoded)
purity = float(mat[0].max() + mat[1].max() + mat[2].max()) / float(mat.sum())
    
pd.crosstab(iris_label, np.array(kmeans.labels_))

col_0,0,1,2
Species,,,
Iris-setosa,50,0,0
Iris-versicolor,3,47,0
Iris-virginica,0,13,37


In [8]:
print "Purity: ", purity, "\n"

Purity:  0.893333333333 



# Perbandingan dengan SKLearn

In [9]:
from sklearn.cluster import KMeans as km

xx = km(init='random', n_clusters=3).fit(iris_data)
mat = confusion_matrix(xx.labels_, iris_label_encoded)
purity = float(mat[0].max() + mat[1].max() + mat[2].max()) / float(mat.sum())

pd.crosstab(iris_label, xx.labels_)

col_0,0,1,2
Species,,,
Iris-setosa,0,50,0
Iris-versicolor,48,0,2
Iris-virginica,14,0,36


In [10]:
print "Purity: ", purity, "\n"

Purity:  0.893333333333 

